# 해당 파일 설명

이 코드를 실행하기 위한 파일, /convert 경로에 다음의 파일이 필요합니다.

1. plant_train_all_weather_feature_add_cluster.csv

이 코드를 모두 실행하고 나면, /model 경로에 다음의 파일이 생성됩니다.

1. model_index.pkl

## 추가 설명

'plant', 'loc',  'tem_in', 'hum_in', 'tem_coil', 'tem_out_loc', 'hum_out_loc', 'cond_loc'

를 가지고 확률을 예측해주는 모델을 만듭니다. 이때 사용되는 모델은 로지스틱 입니다.

# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
# BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 backup_0.96model
data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
lastplus_th_10.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_cond_all_plus_.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.c

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
import lightgbm

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Get the Data

## 1. plant data 가져오기

In [ ]:
# plant 데이터 
data_train = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature_add_cluster.csv"))

# unnamed: 0 제거
data_train.drop(['Unnamed: 0'], axis='columns', inplace=True)

# date 변수 str -> datetime.date()
data_train['date'] = data_train['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 

In [ ]:
# train 데이터 info
data_train = data_train[['plant', 'loc', 'tem_in', 'hum_in', 'tem_coil', 'tem_out_loc', 'hum_out_loc', 'cond_loc']]
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350366 entries, 0 to 350365
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   plant        350366 non-null  float64
 1   loc          350366 non-null  float64
 2   tem_in       350366 non-null  float64
 3   hum_in       350366 non-null  float64
 4   tem_coil     350366 non-null  float64
 5   tem_out_loc  350366 non-null  float64
 6   hum_out_loc  350366 non-null  float64
 7   cond_loc     350366 non-null  float64
dtypes: float64(8)
memory usage: 21.4 MB


In [ ]:
# train 데이터 NULL 제거 후 info
data_train.dropna(inplace=True)
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350366 entries, 0 to 350365
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   plant        350366 non-null  float64
 1   loc          350366 non-null  float64
 2   tem_in       350366 non-null  float64
 3   hum_in       350366 non-null  float64
 4   tem_coil     350366 non-null  float64
 5   tem_out_loc  350366 non-null  float64
 6   hum_out_loc  350366 non-null  float64
 7   cond_loc     350366 non-null  float64
dtypes: float64(8)
memory usage: 24.1 MB


# Modeling


In [ ]:
# 훈련할 모델
model_cond = dict()

## 1. 사용하는 변수 정의

In [ ]:
# 훈련에 사용되는 변수 정의
remove_list = ['cond_loc']
data_columns = list(data_train.columns)
for rmv in remove_list:
  data_columns.remove(rmv)
data_columns

['plant', 'loc', 'tem_in', 'hum_in', 'tem_coil', 'tem_out_loc', 'hum_out_loc']

## 2. 모델 선정

In [ ]:
MLA = [
       #ensemble.AdaBoostClassifier(),
       #ensemble.BaggingClassifier(),
       #ensemble.ExtraTreesClassifier(),
       #ensemble.GradientBoostingClassifier(),
       #ensemble.RandomForestClassifier(),

       #gaussian_process.GaussianProcessClassifier(),

       linear_model.LogisticRegressionCV(),
       #linear_model.PassiveAggressiveClassifier(),
       #linear_model.RidgeClassifierCV(),
       #linear_model.SGDClassifier(),
       #linear_model.Perceptron(),

       #naive_bayes.BernoulliNB(),
       #naive_bayes.GaussianNB(),

       #svm.SVC(probability=True),
       #svm.NuSVC(probability=True),
       #svm.LinearSVC(),

       #tree.DecisionTreeClassifier(),
       #tree.ExtraTreeClassifier(),

       #discriminant_analysis.LinearDiscriminantAnalysis(),
       #discriminant_analysis.QuadraticDiscriminantAnalysis(),

       #XGBClassifier(),
       #lightgbm.LGBMClassifier()
]
MLA

[LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                      fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                      max_iter=100, multi_class='auto', n_jobs=None,
                      penalty='l2', random_state=None, refit=True, scoring=None,
                      solver='lbfgs', tol=0.0001, verbose=0)]

## 3. Training

In [ ]:
Targets = ['cond_loc']

data_train_bin = data_columns

cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=.3, train_size=.6, random_state=0)  ############### 고민해야됨!!!!!!!!!!!

MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']

MLA_compares = list()
mla_index = 0
for Target in Targets:
  MLA_compare = pd.DataFrame(columns=MLA_columns)
  MLA_compares.append(MLA_compare)


  row_index = 0
  for alg in MLA:
    print(alg)
    data = copy.deepcopy(data_train)
    MLA_predict = data[Target]
    MLA_name = alg.__class__.__name__
    MLA_compares[mla_index].loc[row_index, 'MLA Name'] = MLA_name
    MLA_compares[mla_index].loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    cv_results = model_selection.cross_validate(alg, data[data_train_bin], data[Target], cv=cv_split)
    MLA_compares[mla_index].loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compares[mla_index].loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
    MLA_compares[mla_index].loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3

    alg.fit(data[data_train_bin], data[Target])
    MLA_predict[MLA_name] = alg.predict(data[data_train_bin])
    model_cond[(MLA_name, Target)] = copy.deepcopy(alg)
    row_index += 1
  MLA_compares[mla_index].sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
  mla_index += 1 

for idx in range(len(Targets)):
  print(MLA_compares[idx])

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)
               MLA Name  ... MLA Time
0  LogisticRegressionCV  ...  102.346

[1 rows x 5 columns]


## 4. 결과 확인

In [ ]:
print('-------cond_loc-----------')
print('-------------------------')
MLA_compares[0]

-------cond_loc-----------
-------------------------


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': None, '...",0.999834,7.99672e-05,102.346


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

diabetes = datasets.load_diabetes()
X = np.array(data[data_train_bin])
y = np.array(data[Targets])

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     4360.
Date:                Sun, 28 Jun 2020   Prob (F-statistic):               0.00
Time:                        20:36:49   Log-Likelihood:             4.1896e+05
No. Observations:              350366   AIC:                        -8.379e+05
Df Residuals:                  350360   BIC:                        -8.378e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0507      0.001    -85.152      0.0

 1   tem_in      

 2   hum_in      

 3   tem_coil    

 4   tem_out_loc  

 5   hum_out_loc  


## 5. 모델 저장하기

In [ ]:
joblib.dump(model_cond, os.path.join(MODEL_PATH, "model_index.pkl"))

['/content/gdrive/My Drive/weather_bigcontest/4_code/model/model_index.pkl']